In [3]:
#!pip install torch

In [12]:
#!pip install torch-geometric
#!pip install -q torch-sparse
#!pip install torch-scatter

  Using cached torch_scatter-2.0.6.tar.gz (21 kB)
  Created wheel for torch-scatter: filename=torch_scatter-2.0.6-cp37-cp37m-macosx_10_9_x86_64.whl size=300695 sha256=2a620cc98b01820923ae7f14efda0928e07f2b83efac805e73f279752eef4ba2
  Stored in directory: /Users/asmisouhail/Library/Caches/pip/wheels/30/6f/0c/dbdc62f111a9f09962331fd1c13b05806d6c2a2340d27fc0dc
Successfully built torch-scatter


# Libraries

In [13]:
from queries import*
from utilities import*
from scipy.spatial import Delaunay
from numpy import linalg
from sklearn import preprocessing
import copy
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

## Data

In [15]:
PATIENT_ID = '1D1'
INPUT_PATH = './data/input/DEEPMEL_' + PATIENT_ID + '/'
OUTPUT_PATH = './data/output/DEEPMEL_' + PATIENT_ID +'/'
PLOT_PATH ='./plots/DEEPMEL_' + PATIENT_ID +'/'

patch_data = pd.read_csv(OUTPUT_PATH + 'patch_data.csv')
patch_edge_data = pd.read_csv(OUTPUT_PATH + 'patch_edge_data.csv')

In [27]:
patch_edge_data.head(3)

,patch_number_1,patch_1,patch_number_2,patch_2,distance
0,0,"(0.0, 3.0)",1,"(0.0, 4.0)",1.0
1,0,"(0.0, 3.0)",6,"(1.0, 3.0)",1.0
2,1,"(0.0, 4.0)",0,"(0.0, 3.0)",1.0


In [18]:
def get_edge_tensor(edge_df):
    
    patch_number_1 = edge_df.patch_number_1.values
    patch_number_2 = edge_df.patch_number_2.values
    
    edges_index = torch.tensor([patch_number_1,patch_number_2])
    
    return edges_index


In [21]:
get_edge_tensor(patch_edge_data).shape

torch.Size([2, 180])

In [24]:
patch_data = patch_data.sort_values('patch_number')

In [25]:
Directory = os.getcwd()
Directory_path = Directory + '/data/output'
files = os.listdir(Directory_path)

# Getting labels 

In [26]:
files

['DEEPMEL_2P1',
 'DEEPMEL_3I1',
 'DEEPMEL_1F1',
 'DEEPMEL_1L1',
 'DEEPMEL_3C1',
 'DEEPMEL_2U2',
 'DEEPMEL_2Z1',
 'DEEPMEL_3J1',
 'DEEPMEL_2S1',
 'DEEPMEL_1J2',
 'DEEPMEL_1E1',
 'DEEPMEL_1O1',
 'DEEPMEL_2Y1',
 '.DS_Store',
 'DEEPMEL_2K1',
 'DEEPMEL_2A1',
 'DEEPMEL_1W1',
 'DEEPMEL_1X2',
 'DEEPMEL_2H1',
 'DEEPMEL_1X3',
 'DEEPMEL_2B1',
 'DEEPMEL_2L1',
 'DEEPMEL_1P1',
 'DEEPMEL_2O1',
 'DEEPMEL_1Y1',
 'DEEPMEL_1V2',
 'DEEPMEL_1S1',
 'DEEPMEL_2E1',
 'pheno_count.csv',
 'DEEPMEL_2W1',
 'DEEPMEL_3N1',
 'DEEPMEL_3D1',
 'DEEPMEL_1K1',
 'DEEPMEL_3M1',
 'DEEPMEL_2T1',
 'DEEPMEL_3G1',
 'DEEPMEL_1C1',
 'DEEPMEL_2U1',
 'DEEPMEL_3F1',
 'DEEPMEL_1O2',
 'DEEPMEL_2V1',
 'DEEPMEL_3E1',
 'nodes_features.csv',
 'DEEPMEL_3J2',
 'DEEPMEL_1J1',
 'DEEPMEL_1X1',
 'DEEPMEL_1R1',
 'DEEPMEL_2D1',
 'DEEPMEL_2B2',
 'DEEPMEL_2K3',
 'DEEPMEL_2G1',
 '.ipynb_checkpoints',
 'DEEPMEL_3P1',
 'DEEPMEL_2C1',
 'clustering_coefficients_statistics.csv',
 'DEEPMEL_1Z2',
 'DEEPMEL_1U1',
 'DEEPMEL_2J1',
 'DEEPMEL_1V1',
 'DEEPMEL_2R1